In [51]:
import pandas as pd
import seaborn as sns

In [53]:
rdata = pd.read_csv('../data/syrian-refugee-counts.csv')

# as1 = rdata.Refugees != '*'
# as2 = rdata.AsylumSeekers != '*'
# rdata = rdata[as1 & as2]

rdata['Refugees'] = rdata['Refugees'].astype(float)
rdata['AsylumSeekers'] = rdata['AsylumSeekers'].astype(float)

# pad with 0s
rdata = rdata.fillna(0)
rdata['T'] = rdata['Refugees'] + rdata['AsylumSeekers']

# only use subset
rdata = rdata[['Destination', 'T', 'Year']]

In [54]:
d = set(rdata.Destination)
y = set(rdata.Year)
p = pd.MultiIndex.from_product([d,y], names=['destinations', 'year'])
rdata.reindex(p)
rdata.head(5)

,Destination,T,Year
0,Lebanon,1200,1968
1,Lebanon,1200,1969
2,Lebanon,500,1979
3,Italy,2,1980
4,Lebanon,500,1980


In [55]:
pop = pd.read_csv('../data/demographics-wb.csv')
pop = pop[['Country Name',  '2011', '2012', '2013','2014']]
pop.columns = ['Destination', '2011_pop', '2012_pop', '2013_pop', '2014_pop']

In [56]:
# Find top10
r = rdata
r = r.drop(['Year'], 1)
g = r.groupby('Destination').sum()
top10 = g.sort('T', ascending=False).head(10)
top10

,T
Destination,
Turkey,2393091
Lebanon,2134927
Jordan,1452322
Iraq,528157
Egypt,283099
Germany,256993
Sweden,137136
Netherlands,51502
Libya,39428


In [57]:
rd = rdata[rdata['Year'] > 2010]
d_pivot = pd.pivot_table(rd, columns=['Destination', 'Year'], values=['T'])
df = d_pivot.unstack()
# to output:
df.reset_index().to_json(orient='records', path_or_buf='../app/src/data/stats.json')

In [72]:
rfs = df.reset_index()
rfs = rfs.merge(pop, on='Destination', how='left')
rfs['rate_avg'] = rfs[2014] / rfs['2014_pop']
rfs = rfs.sort(2014, ascending=0)
rfs.columns = ['level_0', 'Destination', 'ref_2011', 'ref_2012', 'ref_2013', 'ref_2014', 'pop_2011',
       'pop_2012', 'pop_2013', 'pop_2014', 'rate_avg']
rfs.head()

,level_0,Destination,ref_2011,ref_2012,ref_2013,ref_2014,pop_2011,pop_2012,pop_2013,pop_2014,rate_avg
125,T,Turkey,224.0,248666,585711,1558149.0,73058638,73997128,74932641,75837020,0.020546
62,T,Jordan,2811.0,239289,585304,623112.0,6181000,6318000,6460000,6607000,0.094311
68,T,Lebanon,245.5,127181,851615,573955.5,4382790,4424888,4467390,4510301,0.127254
56,T,Iraq,1714.0,64771,213973,235421.0,31760020,32578209,33417476,34278364,0.006868
41,T,Egypt,94.0,12836,131659,138381.0,79392466,80721874,82056378,83386739,0.001660


In [73]:
rfs.sort('ref_2014', ascending=0).to_json(orient='records', path_or_buf='../app/src/data/stats.json')

In [76]:
rfs[:5]['ref_2014'].sum() / rfs['ref_2014'].sum()

0.92322523243142129

In [77]:
rfs[:5]['ref_2014'].sum()

3129018.5

0.9232252324314213

In [75]:
 rfs[:5]['ref_2014'].sum() / rfs['ref_2014'].sum()

0.92322523243142129

In [49]:
rdata = pd.read_csv('../data/syrian-refugee-counts.csv')

# rdata[rdata.Refugees == '*']['Refugees'] = ""
# rdata[rdata.AsylumSeekers == '*']['AsylumSeekers'] = "0"

rdata['Refugees'] = rdata['Refugees'].astype(float)
rdata['AsylumSeekers'] = rdata['AsylumSeekers'].astype(float)

In [50]:
rdata[rdata['Year'] == 2014].sort(['Refugees'], ascending=0)

,Year,Destination,Origin,Refugees,AsylumSeekers,ReturnedRefugees,IDPs,ReturnedIDPs,StatelessPersons,Others,Total
1326,2014,Turkey,Syrian Arab Rep.,1557899,250,NaN,NaN,NaN,NaN,NaN,1558149
1272,2014,Lebanon,Syrian Arab Rep.,1147494,417,NaN,NaN,NaN,NaN,NaN,1147911
1264,2014,Jordan,Syrian Arab Rep.,623112,NaN,NaN,NaN,NaN,NaN,NaN,623112
1259,2014,Iraq,Syrian Arab Rep.,234196,1225,NaN,NaN,NaN,NaN,NaN,235422
1242,2014,Egypt,Syrian Arab Rep.,138381,NaN,NaN,NaN,NaN,NaN,NaN,138381
1236,2014,Germany,Syrian Arab Rep.,40994,29591,NaN,NaN,NaN,NaN,NaN,70585
1319,2014,Sweden,Syrian Arab Rep.,34285,19011,NaN,NaN,NaN,NaN,NaN,53296
1275,2014,Libya,Syrian Arab Rep.,18653,NaN,NaN,NaN,NaN,NaN,NaN,18653
1210,2014,Armenia,Syrian Arab Rep.,14994,9,NaN,NaN,NaN,NaN,NaN,15003
1295,2014,Netherlands,Syrian Arab Rep.,8692,2323,NaN,NaN,NaN,NaN,NaN,11015
